In [1]:
###Let's start by importing all needed libraries

from sqlalchemy import create_engine
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import json 
import requests

In [2]:
### 1.- CONNECTION TO THE DATABASE .db

# connection string'

sqlitedb_rel_path = '../data/raw/raw_data_project_m1.db'
conn_str = f'sqlite:///{sqlitedb_rel_path}'

engine = create_engine(conn_str)

In [3]:
# Import initial data from the db:uuid , age , countries...
initial_df = pd.read_sql_query("""
SELECT personal_info.uuid, normalized_job_code ,age , country_code FROM personal_info
INNER JOIN country_info ON personal_info.uuid = country_info.uuid
INNER JOIN career_info ON personal_info.uuid = career_info.uuid;
""", engine)

In [4]:
initial_df.head()

,uuid,normalized_job_code,age,country_code
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,None,61 years old,AT
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,861a9b9151e11362eb3c77ca914172d0,57 years old,AT
2,83127080-da3d-0133-c74f-0a81e8b09a82,None,32 years old,AT
3,15626d40-db13-0133-ea5c-0a81e8b09a82,049a3f3a2b5f85cb2971ba77ad66e10c,45 years old,AT
4,24954a70-db98-0133-4a64-0a81e8b09a82,f4b2fb1aa40f661488e2782b6d57ad2f,41 years old,AT


In [5]:
### 2.- CONNECTION TO THE API

# Lets prepare the key before the searching:
job_code_unique = initial_df['normalized_job_code'].unique()

In [6]:
len(job_code_unique)

157

In [7]:
job_code_unique[:5]

array([None, '861a9b9151e11362eb3c77ca914172d0',
       '049a3f3a2b5f85cb2971ba77ad66e10c',
       'f4b2fb1aa40f661488e2782b6d57ad2f',
       '27af8700f5577cec835acee2cb90a2ff'], dtype=object)

In [8]:
# Lets get the information from the API
job_titles = {}
#code = "8a7a3f1bce0958ae9f090683e5a925b7"
for code in job_code_unique[1:]:
    results = requests.get(f'http://api.dataatwork.org/v1/jobs/{code}').json()
    job = results['title']
    job_titles[code] = job

In [9]:
# I am always more comfortable working with pandas df...
job_titles_df = pd.DataFrame(job_titles.items() ,columns=['normalized_job_code','Job Title'])
job_titles_df.head()

,normalized_job_code,Job Title
0,861a9b9151e11362eb3c77ca914172d0,Automatic Data Processing Planner
1,049a3f3a2b5f85cb2971ba77ad66e10c,Data Coordinator
2,f4b2fb1aa40f661488e2782b6d57ad2f,Database Developer
3,27af8700f5577cec835acee2cb90a2ff,Data Entry Specialist
4,c1b670eba9ccb65e7c99f7da116d5b9c,Database Architect


In [10]:
### 3.- WEBSCRAPING TO GET THE COUNTRY CODES:

url = 'https://ec.europa.eu/eurostat/statistics-explained/index.php/Glossary:Country_codes'
html = requests.get(url).content
html[0:600]

b'\xef\xbb\xbf<!DOCTYPE html>\n<html lang="en" dir="ltr" class="client-nojs">\n<head>\n<meta http-equiv="X-UA-Compatible" content="Ie=Edge,chrome=1"/> <meta charset="UTF-8" /><title>Glossary:Country codes - Statistics Explained</title>\n<meta name="generator" content="MediaWiki 1.22.5" />\n<link rel="shortcut icon" href="https://ec.europa.eu/eurostat/estat-theme/images/favicon.ico" />\n<link rel="search" type="application/opensearchdescription+xml" href="/eurostat/statistics-explained/opensearch_desc.php" title="Statistics Explained (en)" />\n<link rel="EditURI" type="application/rsd+xml" href="https://ec.europ'

In [12]:
countries_raw = soup.find('table',{'width':'80%'}).text
countries_raw

'\n\nBelgium\n\n(BE)\n\nGreece\n\n(EL)\n\nLithuania\n\n(LT)\n\nPortugal\n\n(PT)\n\n\nBulgaria\n\n(BG)\n\nSpain\n\n(ES)\n\nLuxembourg\n\n(LU)\n\nRomania\n\n(RO)\n\n\nCzechia\n\n(CZ)\n\nFrance\n\n(FR)\n\nHungary\n\n(HU)\n\nSlovenia\n\n(SI)\n\n\nDenmark\n\n(DK)\n\nCroatia\n\n(HR)\n\nMalta\n\n(MT)\n\nSlovakia\n\n(SK)\n\n\nGermany\n\n(DE)\n\nItaly\n\n(IT)\n\nNetherlands\n\n(NL)\n\nFinland\n\n(FI)\n\n\nEstonia\n\n(EE)\n\nCyprus\n\n(CY)\n\nAustria\n\n(AT)\n\nSweden\n\n(SE)\n\n\nIreland\n\n(IE)\n\nLatvia\n\n(LV)\n\nPoland\n\n(PL)\n'

In [13]:
countries_raw = countries_raw.replace("\n","")

In [14]:
countries_raw

'Belgium(BE)Greece(EL)Lithuania(LT)Portugal(PT)Bulgaria(BG)Spain(ES)Luxembourg(LU)Romania(RO)Czechia(CZ)France(FR)Hungary(HU)Slovenia(SI)Denmark(DK)Croatia(HR)Malta(MT)Slovakia(SK)Germany(DE)Italy(IT)Netherlands(NL)Finland(FI)Estonia(EE)Cyprus(CY)Austria(AT)Sweden(SE)Ireland(IE)Latvia(LV)Poland(PL)'

In [15]:
countries_raw = countries_raw.split(')')

In [16]:
countries_raw

['Belgium(BE',
 'Greece(EL',
 'Lithuania(LT',
 'Portugal(PT',
 'Bulgaria(BG',
 'Spain(ES',
 'Luxembourg(LU',
 'Romania(RO',
 'Czechia(CZ',
 'France(FR',
 'Hungary(HU',
 'Slovenia(SI',
 'Denmark(DK',
 'Croatia(HR',
 'Malta(MT',
 'Slovakia(SK',
 'Germany(DE',
 'Italy(IT',
 'Netherlands(NL',
 'Finland(FI',
 'Estonia(EE',
 'Cyprus(CY',
 'Austria(AT',
 'Sweden(SE',
 'Ireland(IE',
 'Latvia(LV',
 'Poland(PL',
 '']

In [17]:
list_countries = [i.split('(') for i in countries_raw]
country_codes_df = pd.DataFrame(list_countries, columns=('Country','country_code'))

In [69]:
#Life is better with a df...
country_codes_df

,Country,country_code
0,Belgium,BE
1,Greece,EL
2,Lithuania,LT
3,Portugal,PT
4,Bulgaria,BG
5,Spain,ES
6,Luxembourg,LU
7,Romania,RO
8,Czechia,CZ
9,France,FR


In [70]:
final_df = pd.merge(initial_df,  
                      job_titles_df,  
                      on ='normalized_job_code',  
                      how ='inner') 

In [71]:
definitive_df = pd.merge(final_df,  
                      country_codes_df,  
                      on ='country_code',  
                      how ='inner') 

In [72]:
definitive_df.head()

,uuid,normalized_job_code,age,country_code,Job Title,Country
0,54f0f1c0-dda1-0133-a559-0a81e8b09a82,861a9b9151e11362eb3c77ca914172d0,57 years old,AT,Automatic Data Processing Planner,Austria
1,cf24ed60-da3f-0133-0034-0a81e8b09a82,861a9b9151e11362eb3c77ca914172d0,48 years old,AT,Automatic Data Processing Planner,Austria
2,15626d40-db13-0133-ea5c-0a81e8b09a82,049a3f3a2b5f85cb2971ba77ad66e10c,45 years old,AT,Data Coordinator,Austria
3,ca3f65f0-d92b-0133-3959-0a81e8b09a82,049a3f3a2b5f85cb2971ba77ad66e10c,46 years old,AT,Data Coordinator,Austria
4,24954a70-db98-0133-4a64-0a81e8b09a82,f4b2fb1aa40f661488e2782b6d57ad2f,41 years old,AT,Database Developer,Austria


In [73]:
def fix_age (age):
    if 'years old' in age:
        born_y = 2016 - int(re.sub("[^\d]", "", age)) 
    else:
        born_y = int(age)
        
    right_age = 2021 - born_y     
    
    return right_age 

definitive_df['age'] = definitive_df.apply(lambda x: fix_age(x['age']), axis=1 )

In [74]:
definitive_df.head()

,uuid,normalized_job_code,age,country_code,Job Title,Country
0,54f0f1c0-dda1-0133-a559-0a81e8b09a82,861a9b9151e11362eb3c77ca914172d0,62,AT,Automatic Data Processing Planner,Austria
1,cf24ed60-da3f-0133-0034-0a81e8b09a82,861a9b9151e11362eb3c77ca914172d0,53,AT,Automatic Data Processing Planner,Austria
2,15626d40-db13-0133-ea5c-0a81e8b09a82,049a3f3a2b5f85cb2971ba77ad66e10c,50,AT,Data Coordinator,Austria
3,ca3f65f0-d92b-0133-3959-0a81e8b09a82,049a3f3a2b5f85cb2971ba77ad66e10c,51,AT,Data Coordinator,Austria
4,24954a70-db98-0133-4a64-0a81e8b09a82,f4b2fb1aa40f661488e2782b6d57ad2f,46,AT,Database Developer,Austria


In [75]:
def group_age (age):
    if age <= 25:
         age_group = '(18-25)' 
    elif age <= 35:
        age_group = '[30-35)'
    elif age <= 45:
        age_group = '[40-45)'
    elif age <= 55:
        age_group = '[45-55)'
    else:
        age_group = '[55-70)'
    
    return age_group

definitive_df['Age Group'] = definitive_df.apply(lambda x: group_age(x['age']), axis=1 )

In [76]:
definitive_df.head()

,uuid,normalized_job_code,age,country_code,Job Title,Country,Age Group
0,54f0f1c0-dda1-0133-a559-0a81e8b09a82,861a9b9151e11362eb3c77ca914172d0,62,AT,Automatic Data Processing Planner,Austria,[55-70)
1,cf24ed60-da3f-0133-0034-0a81e8b09a82,861a9b9151e11362eb3c77ca914172d0,53,AT,Automatic Data Processing Planner,Austria,[45-55)
2,15626d40-db13-0133-ea5c-0a81e8b09a82,049a3f3a2b5f85cb2971ba77ad66e10c,50,AT,Data Coordinator,Austria,[45-55)
3,ca3f65f0-d92b-0133-3959-0a81e8b09a82,049a3f3a2b5f85cb2971ba77ad66e10c,51,AT,Data Coordinator,Austria,[45-55)
4,24954a70-db98-0133-4a64-0a81e8b09a82,f4b2fb1aa40f661488e2782b6d57ad2f,46,AT,Database Developer,Austria,[45-55)


In [77]:
grouped_df = definitive_df[['uuid','Country', 'Job Title','Age Group']].groupby(['Country', 'Job Title','Age Group']).count()
grouped_df.rename(columns={'uuid':'Quantity'}, inplace=True)
grouped_df['Percentage'] = grouped_df.groupby(level=0).transform(lambda x: x/x.sum())

In [78]:
grouped_df.head()

Quantity  \
Country Job Title                                          Age Group             
Austria Automatic Data Processing Customer Liaison (ADP... [40-45)           2   
        Automatic Data Processing Planner                  [45-55)           1   
                                                           [55-70)           1   
        Automatic Data Processing Systems Security Spec... [30-35)           1   
                                                           [45-55)           1   

                                                                      Percentage  
Country Job Title                                          Age Group              
Austria Automatic Data Processing Customer Liaison (ADP... [40-45)      0.027027  
        Automatic Data Processing Planner                  [45-55)      0.013514  
                                                           [55-70)      0.013514  
        Automatic Data Processing Systems Security Spec... [30-35)      0.013514  
                                                           [45-55)      0.013514

In [79]:
grouped_df = grouped_df.reset_index()

In [80]:
grouped_df.head()

,Country,Job Title,Age Group,Quantity,Percentage
0,Austria,Automatic Data Processing Customer Liaison (AD...,[40-45),2,0.027027
1,Austria,Automatic Data Processing Planner,[45-55),1,0.013514
2,Austria,Automatic Data Processing Planner,[55-70),1,0.013514
3,Austria,Automatic Data Processing Systems Security Spe...,[30-35),1,0.013514
4,Austria,Automatic Data Processing Systems Security Spe...,[45-55),1,0.013514


In [66]:
country = "Spain"

In [81]:
if country == "All":
    filtered_country_df = analyzed_df
else:
    filtered_country_df = grouped_df[grouped_df['Country']== country]

NameError: name 'country' is not defined

In [66]:
format_dict = {'Percentage': '{:.2%}'}
grouped_df = grouped_df.style.format(format_dict)

In [67]:
type(grouped_df)

pandas.io.formats.style.Styler

In [68]:
    if country == "All":
        filtered_country_df = analyzed_df
    else:
        filtered_country_df = analyzed_df[analyzed_df['Country']== country]

,Country,Job Title,Age Group,Quantity,Percentage
0,Spain,Analytical Data Miner,[30-35),1,0.17%
1,Spain,Automatic Data Processing Customer Liaison (ADP Customer Liaison),(18-25),1,0.17%
2,Spain,Automatic Data Processing Customer Liaison (ADP Customer Liaison),[30-35),2,0.34%
3,Spain,Automatic Data Processing Customer Liaison (ADP Customer Liaison),[40-45),2,0.34%
4,Spain,Automatic Data Processing Customer Liaison (ADP Customer Liaison),[45-55),4,0.69%
5,Spain,Automatic Data Processing Customer Liaison (ADP Customer Liaison),[55-70),2,0.34%
6,Spain,Automatic Data Processing Planner,[30-35),1,0.17%
7,Spain,Automatic Data Processing Planner,[40-45),4,0.69%
8,Spain,Automatic Data Processing Planner,[45-55),3,0.52%
9,Spain,Automatic Data Processing Planner,[55-70),1,0.17%
